In [1]:
import subprocess
import matplotlib.pyplot
import os
import matplotlib.pyplot as plt
rootdir = '/home/ubuntu/spect_files/'

In [2]:
spectograms = []
spect_read = []
spectograms_ids = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('png'): 
            x = plt.imread(subdir+'/'+file)
            if str(x.shape) == '(513, 800, 3)': 
                spect_read.append(x)
                name = subdir.replace(rootdir, '')
                name = name.replace('/spects', "")
                spectograms_ids.append(name)
                spectograms.append(file)
x_train = spect_read
y_train = spectograms_ids

In [3]:
import random
test_indices = random.sample(range(len(x_train)), 729)
train_indices = []
for i in range(len(x_train)):
    if i not in test_indices:
        train_indices.append(i)

x_new_train = []
y_new_train = []
x_test = []
y_test = []

for i in test_indices:
    x_test.append(x_train[i])
    y_test.append(y_train[i])


for i in train_indices:
    x_new_train.append(x_train[i])
    y_new_train.append(y_train[i])

In [4]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
encoder = LabelEncoder()
y_temp_train = y_new_train
encoder.fit(y_temp_train)
encoded_Y = encoder.transform(y_temp_train)
dummy_y = np_utils.to_categorical(encoded_Y)

In [6]:
y_temp2_train = y_test
encoder.fit(y_temp2_train)
encoded_Y = encoder.transform(y_temp2_train)
dummy2_y = np_utils.to_categorical(encoded_Y)

In [7]:
from keras import backend as K

In [8]:
K.image_data_format()

'channels_last'

In [9]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import optimizers
from keras.layers.advanced_activations import ELU, PReLU, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten

# New NN

In [10]:
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(513, 800, 3)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(10))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 513, 800, 8)       224       
_________________________________________________________________
activation_1 (Activation)    (None, 513, 800, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 511, 798, 8)       584       
_________________________________________________________________
activation_2 (Activation)    (None, 511, 798, 8)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 399, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 255, 399, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 255, 399, 16)      1168      
__________

In [12]:
from keras.callbacks import ModelCheckpoint
filepath="/home/ubuntu/weights.best.hdf5"
model.fit(np.array(x_new_train), dummy_y,
          batch_size=30,
          epochs=5,
          verbose=1, 
         validation_data=(np.array(x_test[:700]), dummy2_y[:700]))
model.save_weights('my_model_weights.h5')

Train on 1864 samples, validate on 700 samples
Epoch 1/5
1864/1864 [==============================] - 95s 51ms/step - loss: 0.9519 - acc: 0.7420 - val_loss: 0.3306 - val_acc: 0.9486
Epoch 2/5
1864/1864 [==============================] - 77s 41ms/step - loss: 0.2270 - acc: 0.9405 - val_loss: 0.1349 - val_acc: 0.9771
Epoch 3/5
1864/1864 [==============================] - 77s 41ms/step - loss: 0.1083 - acc: 0.9769 - val_loss: 0.1354 - val_acc: 0.9843
Epoch 4/5
1864/1864 [==============================] - 77s 41ms/step - loss: 0.1091 - acc: 0.9759 - val_loss: 0.0816 - val_acc: 0.9929
Epoch 5/5
1864/1864 [==============================] - 77s 41ms/step - loss: 0.0794 - acc: 0.9812 - val_loss: 0.2323 - val_acc: 0.9414


In [17]:
r = 707
print(model.predict_classes(np.expand_dims(np.array(x_test[r]), axis=0)))
dummy2_y[r]

1/1 [==============================] - 0s 20ms/step
[1]


array([ 0.,  1.,  0.,  0.,  0.,  0.])

# Old NN

In [55]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(513, 800, 3)))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(16, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(40, activation='relu'))
model.add(Dense(7, activation='softmax'))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 511, 798, 64)      1792      
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 510, 797, 32)      8224      
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 170, 265, 32)      0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 169, 264, 32)      4128      
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 56, 88, 32)        0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 55, 87, 16)        2064      
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 18, 29, 16)        0         
__________

In [58]:
model.fit(np.array(x_new_train), dummy_y,
          batch_size=30,
          epochs=5,
          verbose=1,
          validation_data=(np.array(x_test), dummy2_y))

Train on 2067 samples, validate on 729 samples
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[30,32,170,265]
	 [[Node: training_2/SGD/gradients/max_pooling2d_59/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@max_pooling2d_59/MaxPool"], data_format="NHWC", ksize=[1, 3, 3, 1], padding="VALID", strides=[1, 3, 3, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_79/Relu, max_pooling2d_59/MaxPool, training_2/SGD/gradients/conv2d_80/convolution_grad/Conv2DBackpropInput)]]

Caused by op 'training_2/SGD/gradients/max_pooling2d_59/MaxPool_grad/MaxPoolGrad', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/u